# TODOs (project)

- Add NA values (see "data.archive.yaml")
- Add lab_ids for material sources
- Add date when we have change ISO format (see https://github.com/Automate-Solar/BERTHA/pull/11)


In [1]:
import json
import yaml
import zipfile

In [2]:
with open('data/data.json') as f:
    data = json.load(f)

In [3]:
dic = data[0]

In [4]:
#dic

In [5]:
# print(yaml.dump(steps))

In [6]:
def getPoweredAxes(arr):
    axes = []
    for i in range(len(arr)):
        if arr[i]:
            axes.append(str(i+1))
    return axes

In [7]:
def getYamlMagnetrons(dic, power_axes):
    axes = getPoweredAxes(power_axes)
    return [getYamlMagnetron(dic, axis) for axis in axes]

In [8]:
def getYamlMagnetron(dic, axis):
    magnetron = {
        'name' : dic['Material Ax' + axis],
        'material' : {
            'name' : dic['Material Ax' + axis],
            'lab_id' : None
        },
        'vapor_source' : {
            'setpoints' : {
            'set_power' : dic['Power_Ax' + axis + '_setpoint_[W]'],
            'set_voltage' : None,
            'set_current' : None
            },
            'power' : {
                'values' : dic['Actual_Power_Ax' + axis + '_[W]'],
                'times' : dic['Time_[s]'],
                'mean' : dic['Actual_Power_Ax' + axis + '_[W]_Mean'],
                'error' : dic['Actual_Power_Ax' + axis + '_[W]_STD']
            },
            'voltage' : {
                'values' : dic['Voltage_Ax' + axis + '_[V]'],
                'times' : dic['Time_[s]'],
                'mean' : dic['Voltage_Ax' + axis + '_[V]_Mean'],
                'error' : dic['Voltage_Ax' + axis + '_[V]_STD'],
            },
            'current' : {
                'values' : [],
                'times' : dic['Time_[s]'],
                'mean' : None,
                'error' : None
            },
            'power_supply' : {
                'instrument_id' : "Power Supply Ax" + axis,
                'frequency' : None
            }
        },
        'position' : {
            'center_xyz' : None,
            'center_normal' : None,
            'rotation' : None
        }
    }
    return magnetron

In [9]:
arr = [True, True, True]

        
#getYamlMagnetrons(dic, arr)
        

# Environment

In [10]:
def getYamlQCM(dictionary, i):
    prefix = 'QCM_' + str(i) + '_'
    yamlQCM = {
            'crystal_info' : {
                'info' : None,
                'resonant_frequency' : None
            },
            'sensor_data' : {
                'values' : dictionary[prefix + 'frequency_[Hz]'],
                'mean' : None,
                'error' : None,
                'slope' : dictionary[prefix + 'rate_[df/dt]']
            },
            'remaining_lifetime' : {
                'value' : dictionary[prefix + 'xtal_lifetime_[%]']
            },
            'mass_deposition_rate' : {
                'value' : None,
                'error' : None
            }
        }
    return yamlQCM

In [11]:
def getYamlEnv(dictionary):
    environment = {
        'gas_flow' : {
            'values' :  dictionary['Gas_flow_[sccm]'],
            'mean' : dictionary['Gas_flow_[sccm]_Mean'],
            'error' : dictionary['Gas_flow_[sccm]_STD'],
            'gas' : 'Here we do the pubchem thing'
        },
        'setpoints' : {
            'pressure' : dictionary['Pressure_setpoint_[mTorr]'],
            'flow' : None,
        },
        'pressure' : {
            'values' : dictionary['Actual_pressure_[mTorr]'],
            'mean' : dictionary['Actual_pressure_[mTorr]_Mean'],
            'error' : dictionary['Actual_pressure_[mTorr]_STD']
        },
        'sensors' : [getYamlQCM(dictionary, i+1) for i in range(3)]
    }
    return environment
    


In [12]:
#environment = getYamlEnv(dic)

In [13]:
data = {'data' : {
            'm_def' : "../uploads/1WE7CfSoQbS8LidbT0XLfw/raw/automate-solar_schema.archive.yaml#/definitions/section_definitions/AutomateSolarSputterDeposition",
            'quantity' : None,
            'name' : dic['Run_ID'],
            'lab_id' : "Uppsala University Åutomate-Solar",
            'description' : f'{dic["Campaign description"]}, {dic["Campaign code"]}, {dic["Series_description"]}',
            'steps' : [
                {
                    'sources' : getYamlMagnetrons(dic, [True,True,True]),
                    'environment' : getYamlEnv(dic)
                }
            ]
    }
}

In [14]:
#print(yaml.dump(data, default_flow_style=False))

In [15]:
with open('data/data.archive.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [16]:
with zipfile.ZipFile('data/data.zip', 'w') as zipped_f:
    zipped_f.writestr("data/data.archive.yaml", yaml.dump(data, default_flow_style=False))

In [17]:
#test = yaml.dump(data, default_flow_style=False)